# Import

In [25]:
import numpy as np
import pandas as pd

# Data preporation

In [26]:
# Load data of previous day
file_name = 'release/covid19-russia-cases-scrf.csv'
rus_df = pd.read_csv(file_name)
rus_df.tail()

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
4242,2020-05-23,Челябинская область,Chelyabinsk region,74.0,80.0,0.0,29.0,2073.0,11.0,754.0
4243,2020-05-23,Чеченская Республика,Chechen Republic,95.0,20.0,1.0,16.0,1066.0,12.0,659.0
4244,2020-05-23,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,2.0,1.0,6.0,58.0,1.0,30.0
4245,2020-05-23,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,36.0,0.0,288.0,2025.0,5.0,686.0
4246,2020-05-23,Ярославская область,Yaroslavl region,76.0,115.0,0.0,49.0,2424.0,13.0,493.0


In [27]:
# Read update data
# Result of data_parsing_SCRF.ipynb
file_name = f'covid19-russia-cases-upd-scrf.csv'
upd_df = pd.read_csv(file_name)
upd_df.sort_values('Confirmed', ascending=False).head()

,Region/City,Confirmed,Recovered,Deaths,Date,Day-Confirmed,Day-Deaths,Day-Recovered
29,Москва,163913,49840,1993,2020-05-24,2516.0,59.0,2427.0
30,Московская область,33515,5520,329,2020-05-24,862.0,5.0,357.0
63,Санкт-Петербург,13339,3452,133,2020-05-24,384.0,20.0,328.0
32,Нижегородская область,8047,2398,65,2020-05-24,221.0,5.0,141.0
46,Республика Дагестан,4095,3267,84,2020-05-24,113.0,14.0,265.0


In [28]:
# Strip text data
rus_df['Region/City'] = rus_df['Region/City'].astype('str').str.strip('\u200b') 
upd_df['Region/City'] = upd_df['Region/City'].astype('str').str.strip('\u200b')

# Date to datetime
upd_df['Date'] = pd.to_datetime(upd_df['Date'])
rus_df['Date'] = pd.to_datetime(rus_df['Date'])

# Updating

In [30]:
# Add region ID and Eng columns
regions_df = pd.read_csv('release/regions-info.csv')

right = regions_df[['Region_ID', 'Region', 'Region_eng']]
upd_df = upd_df.merge(right, left_on='Region/City', right_on='Region', how='left')
upd_df['Region/City-Eng'] = upd_df.Region_eng
upd_df = upd_df[['Date', 'Region/City', 'Region/City-Eng', 'Region_ID', 'Day-Confirmed', 'Day-Deaths', 'Day-Recovered', 'Confirmed', 'Deaths', 'Recovered']]

In [31]:
rus_df = pd.concat([rus_df, upd_df]).reset_index(drop=True)
rus_df.tail(10)

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
4321,2020-05-24,Тюменская область,Tyumen region,72.0,45.0,0.0,34.0,1411.0,7.0,625.0
4322,2020-05-24,Удмуртская Республика,Udmurt republic,18.0,18.0,0.0,1.0,510.0,12.0,324.0
4323,2020-05-24,Ульяновская область,Ulyanovsk region,73.0,97.0,0.0,36.0,1861.0,6.0,498.0
4324,2020-05-24,Хабаровский край,Khabarovsk region,27.0,61.0,0.0,13.0,1633.0,10.0,691.0
4325,2020-05-24,Ханты-Мансийский АО,Khanty-Mansiysk,86.0,102.0,2.0,34.0,1670.0,17.0,579.0
4326,2020-05-24,Челябинская область,Chelyabinsk region,74.0,62.0,1.0,65.0,2135.0,12.0,819.0
4327,2020-05-24,Чеченская Республика,Chechen Republic,95.0,24.0,0.0,1.0,1090.0,12.0,660.0
4328,2020-05-24,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,1.0,0.0,2.0,59.0,1.0,32.0
4329,2020-05-24,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,29.0,1.0,34.0,2054.0,6.0,720.0
4330,2020-05-24,Ярославская область,Yaroslavl region,76.0,107.0,0.0,10.0,2531.0,13.0,503.0


In [32]:
# Negative values fix
# Negative values can appear if some cases was removed from statistic
def negval_fix(reg):
    last = reg.iloc[-1]
    diff_c = last['Day-Confirmed']*(-1) if last['Day-Confirmed'] < 0 else 0
    diff_d = last['Day-Deaths']*(-1) if last['Day-Deaths'] < 0 else 0
    diff_r = last['Day-Recovered']*(-1) if last['Day-Recovered'] < 0 else 0
    
    last['Day-Confirmed'] += diff_c
    last['Day-Deaths']    += diff_d
    last['Day-Recovered'] += diff_r
    reg.iloc[-1] = last
    
    if diff_c + diff_d + diff_r > 0:
        print(diff_c, diff_d, diff_r, last['Region/City'])
        prev = reg.iloc[-2]
        prev['Confirmed'] -= diff_c
        prev['Deaths']    -= diff_d
        prev['Recovered'] -= diff_r
        reg.iloc[-2] = prev
    
    return reg
    
rus_df = rus_df.groupby('Region/City').apply(negval_fix).reset_index(drop=True)

In [33]:
# Manual fixes if needed
# rus_df.loc[287, 'Confirmed'] = 5
# rus_df.loc[321, 'Confirmed'] = 5
# rus_df.loc[359, 'Day-Confirmed'] = 0

# rus_df.loc[3775, 'Deaths'] = 7
# rus_df.loc[3775, 'Day-Deaths'] = 0

# Data checking

In [34]:
rus_sum = rus_df.loc[rus_df['Region/City'] != 'Diamond Princess']

print('Russia Confirmed:', rus_sum.groupby('Region/City')['Confirmed'].max().sum(),
      '| By day:', rus_sum.groupby('Region/City')['Day-Confirmed'].sum().sum())
print('Russia Deaths:', rus_sum.groupby('Region/City')['Deaths'].max().sum(),
      '| By day:', rus_sum.groupby('Region/City')['Day-Deaths'].sum().sum())
print('Russia Recovered:', rus_sum.groupby('Region/City')['Recovered'].max().sum(),
      '| By day:', rus_sum.groupby('Region/City')['Day-Recovered'].sum().sum())

rus_regs = rus_sum.groupby('Region/City')['Confirmed'].max().reset_index()
rus_regs['Confirmed-ByDay'] = rus_sum.groupby('Region/City')['Day-Confirmed'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Confirmed'] != rus_regs['Confirmed-ByDay']])

rus_regs = rus_sum.groupby('Region/City')['Recovered'].max().reset_index()
rus_regs['Recovered-ByDay'] = rus_sum.groupby('Region/City')['Day-Recovered'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Recovered'] != rus_regs['Recovered-ByDay']])

rus_regs = rus_sum.groupby('Region/City')['Deaths'].max().reset_index()
rus_regs['Deaths-ByDay'] = rus_sum.groupby('Region/City')['Day-Deaths'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Deaths'] != rus_regs['Deaths-ByDay']])

# rus_regs.to_csv('rus_regs.csv')

Russia Confirmed: 344481.0 | By day: 344481.0
Russia Deaths: 3545.0 | By day: 3541.0
Russia Recovered: 113299.0 | By day: 113299.0
Empty DataFrame
Columns: [Region/City, Confirmed, Confirmed-ByDay]
Index: []
Empty DataFrame
Columns: [Region/City, Recovered, Recovered-ByDay]
Index: []
             Region/City  Deaths  Deaths-ByDay
9    Воронежская область    16.0          14.0
34  Новгородская область     7.0           5.0


# Data saving

In [35]:
file_name = 'release/covid19-russia-cases-scrf.csv'
rus_df.to_csv(file_name, index=False)